audeering json 파일 전처리 코드

In [ ]:
import json

# Colab 환경의 파일 경로 지정 (예: /content/your_file.json)
file_path = "/content/emotion_results-audeering.json"

# 원본 JSON 데이터 로드
with open(file_path, "r") as file:
    original_data = json.load(file)

# 파일 이름에서 숫자를 추출하여 정렬하는 함수 정의
def sort_by_file_name(data):
    return sorted(data, key=lambda x: int(x["original_file"].split(".")[0]))

# 데이터 정렬
sorted_data = sort_by_file_name(original_data)

# 변환된 데이터를 저장할 리스트
transformed_data = []

# 데이터 변환
for entry in sorted_data:
    transformed_entry = {}
    base_name = entry["original_file"].split(".")[0]
    transformed_entry[f"{base_name}_predictions"] = entry["original_predictions"]
    transformed_entry[f"{base_name}n_predictions"] = entry["noise_predictions"]
    transformed_entry[f"{base_name}nn_predictions"] = entry["noise2_predictions"]
    transformed_data.append(transformed_entry)

# 변환된 데이터를 새 파일로 저장
output_file_path = "/content/emotion_results-audeering2.json"
with open(output_file_path, "w") as file:
    json.dump(transformed_data, file, indent=4)

print(f"변환 완료: {output_file_path} 파일로 저장되었습니다.")


변환 완료: /content/emotion_results-audeering2.json 파일로 저장되었습니다.


다른 파일 전처리 코드

In [ ]:
import json

# JSON 파일 경로
# 파일 경로 적을 것
input_file_path = "/content/emotion_results-kor-kresnik.json"
output_file_path = "/content/emotion_results-kor-kresnik2.json"

# JSON 파일 로드
with open(input_file_path, "r") as file:
    original_data = json.load(file)

# 파일 이름 정렬 함수 (숫자 정렬을 고려)
def sort_file_names(data):
    return sorted(data.items(), key=lambda x: int(x[0].split('.')[0].replace('n', '').replace('nn', '')))

# 데이터를 정렬 및 변환
sorted_data = sort_file_names(original_data)

transformed_data = []
for file_name, content in sorted_data:
    base_name = file_name.split('.')[0]  # 파일 이름에서 확장자 제거
    key_suffix = (
        "" if "nn" not in file_name and "n" not in file_name
        else "n" if "nn" not in file_name
        else "nn"
    )
    key_name = f"{base_name}{key_suffix}_predictions"
    transformed_data.append({key_name: list(content["emotions"].values())})

# 결과 저장
with open(output_file_path, "w") as file:
    json.dump(transformed_data, file, indent=4)

print(f"변환 완료: {output_file_path} 파일로 저장되었습니다.")


ValueError: invalid literal for int() with base 10: '1nn'

In [ ]:
import json
import os
import re

# JSON 파일이 저장된 디렉터리 경로
input_directory = "/content/input"  # 입력 파일 디렉터리
output_directory = "/content/output"  # 출력 파일 디렉터리

# 출력 디렉터리 생성
os.makedirs(output_directory, exist_ok=True)

def normalize_key(key):
    """키를 정상화: nn -> n, nnnn -> nn"""
    if "nnnn" in key:
        return key.replace("nnnn", "nn")
    elif "nn" in key:
        return key.replace("nn", "n")
    return key

def extract_base_number(key):
    """키에서 기본 숫자 부분을 추출"""
    match = re.match(r"(\d+)", key)
    return match.group(1) if match else None

def process_json_file(file_path, output_path):
    with open(file_path, "r") as file:
        data = json.load(file)

    processed_data = {}

    # 데이터 그룹화
    for item in data:
        for key, values in item.items():
            # 키에서 기본 숫자 추출 및 접미사 변환
            base_number = extract_base_number(key)
            normalized_key = normalize_key(key)

            if base_number not in processed_data:
                processed_data[base_number] = {}

            processed_data[base_number][normalized_key] = values

    # 결과를 리스트로 변환
    result = [values for _, values in sorted(processed_data.items(), key=lambda x: int(x[0]))]

    # 변환된 데이터 저장
    with open(output_path, "w") as file:
        json.dump(result, file, indent=4)

# 디렉터리 내 모든 JSON 파일 처리
for filename in os.listdir(input_directory):
    if filename.endswith(".json"):
        input_file_path = os.path.join(input_directory, filename)
        output_file_path = os.path.join(output_directory, filename)
        process_json_file(input_file_path, output_file_path)

print(f"모든 파일이 '{output_directory}' 디렉터리에 저장되었습니다.")


모든 파일이 '/content/output' 디렉터리에 저장되었습니다.
